In [2]:
"""
gas.ipynb

Goal with this notebook is to continue analysis on the gas
market from the end of blocks.ipynb, verifying behavior
seen in blocks.ipynb pre-merge.

Emphasis is on per block gas utilization (i.e. gas_used / gas_limit)
from the introduction of EIP1559 (Aug 2021) to Aug 2022 pre-merge, since
blocks.ipynb only analyzes 1 month of data pre-merge.
"""

import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import typing as tp

from ape import accounts, chain, Contract, networks
from pathlib import Path
from scipy import stats

In [3]:
# SEE: https://gist.github.com/banteg/dcf6082ff7fc6ad51ce220146f29d9ff
networks.parse_network_choice('ethereum:mainnet:alchemy').__enter__()

<alchemy chain_id=1>

In [36]:
# some relevant numbers
# SEE: https://ethereum.org/en/history/
# NOTE: only looking from start 1 month prior to London as care about trends in gas market pre-merge (not post EIP1559)
# SEE: https://decentralizedthoughts.github.io/2022-03-10-eip1559/ for transition to EIP1559 (was a good read for empirical/charts)
start_block_number = 12765616  # ~ 1 month prior to London
london_block_number = 12965000  # EIP1559 introduction (London)
end_block_number = 15343216 # ~ start_block_number from blocks.ipynb

In [39]:
# load EL blocks from csv if already queried, else do the query and save in csv
def load_blocks(fp: str, compression: str, start: int, end: int, step: int) -> pd.DataFrame:
    if os.path.exists(fp):
        return pd.read_csv(fp, compression=compression)
    else:
        # query blocks over timespan
        # chunk via range(start, end, step) to save transactions to temp csv
        tmp_fp = str(Path(fp).parent / Path('_tmp_' + Path(fp).name))
        is_head = True
        for b in range(start, end, step):
            qb = chain.blocks.query('*', start_block=b, stop_block=b+step-1)
            
            # append to temp
            qb.to_csv(tmp_fp, mode='a', compression=compression, index=False, header=is_head)
            if is_head:
                is_head = False

        # once done iterating, mv temp to output file and load
        os.rename(tmp_fp, fp)
        return pd.read_csv(fp, compression=compression)

In [ ]:
# get blocks over 1 year prior to EIP1559 intro (from Aug 2021 to Aug 2022)
# NOTE: query takes around ~Xh if not from csv
step = 7200 # ~ 1d
%time qb = load_blocks('./data/gas/blocks.csv', 'gzip', start_block_number, end_block_number, step)